<a href="https://colab.research.google.com/github/isaacdono/ml-studies/blob/main/recommendation_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo Prático: Sistemas de Recomendação

Sistemas de Recomendação estão por toda parte, desde a Netflix sugerindo filmes até a Amazon recomendando produtos. O objetivo é prever o interesse de um usuário em um item.

Neste notebook, vamos explorar a abordagem mais clássica: a **Filtragem Colaborativa (Collaborative Filtering)**.

A ideia é fazer recomendações com base na "sabedoria das massas":
1.  **User-Based CF**: Encontra usuários com gostos parecidos com os seus e recomenda itens que eles gostaram e você ainda não viu. ("*Usuários parecidos com você também gostaram de...*")
2.  **Item-Based CF**: Encontra itens similares a um que você gostou e os recomenda. ("*Usuários que gostaram deste item também gostaram de...*")

Vamos implementar ambos do zero usando um pequeno conjunto de dados.

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

print("Bibliotecas importadas com sucesso!")

# ====================================================================
# Célula 3: Criando o Conjunto de Dados
# ====================================================================

# Vamos criar um DataFrame simples com avaliações de filmes de usuários.
# As notas vão de 1 a 5. NaN significa que o usuário não avaliou o filme.
data = {
    'Ana': {'Matrix': 5, 'Titanic': 3, 'O Poderoso Chefão': 4, 'Forrest Gump': None, 'Interestelar': 5},
    'Bruno': {'Matrix': 5, 'Titanic': 2, 'O Poderoso Chefão': None, 'Forrest Gump': 3, 'Interestelar': 4},
    'Carla': {'Matrix': 2, 'Titanic': 5, 'O Poderoso Chefão': 2, 'Forrest Gump': 5, 'Interestelar': 2},
    'Daniel': {'Matrix': None, 'Titanic': 4, 'O Poderoso Chefão': 5, 'Forrest Gump': 5, 'Interestelar': None},
    'Elisa': {'Matrix': 4, 'Titanic': None, 'O Poderoso Chefão': 5, 'Forrest Gump': 2, 'Interestelar': 5}
}

ratings_df = pd.DataFrame(data)

print("Matriz de Avaliações (Usuário-Item):")
ratings_df


In [ ]:
"""
### Abordagem 1: Filtragem Colaborativa Baseada no Usuário (User-Based)

**Objetivo**: Recomendar filmes para a `Ana`.

**Passos**:
1.  Encontrar o usuário mais parecido com a `Ana`.
2.  Ver quais filmes esse usuário gostou e que a `Ana` ainda não viu.
"""

# Passo 1: Calcular a similaridade entre usuários
# Primeiro, preenchemos os NaNs com 0 para que o cálculo da similaridade funcione.
# A matriz precisa ser (amostras, features), então usamos .T (transposta)
ratings_filled = ratings_df.fillna(0)
user_similarity = cosine_similarity(ratings_filled.T)

# Transformando em um DataFrame para melhor visualização
user_sim_df = pd.DataFrame(user_similarity, index=ratings_df.columns, columns=ratings_df.columns)

print("Matriz de Similaridade entre Usuários:")
print(user_sim_df)


# Passo 2: Fazer a recomendação para a 'Ana'
target_user = 'Ana'

# Encontra usuários similares (exceto ela mesma)
similar_users = user_sim_df[target_user].sort_values(ascending=False)[1:]
print(f"\nUsuários mais similares à {target_user}:\n{similar_users}")

# Pegando o usuário mais similar
most_similar_user = similar_users.index[0]
print(f"\nO usuário mais similar é: {most_similar_user}")

# Encontrando filmes que o usuário similar avaliou bem e que a Ana não viu
recommendations = []
similar_user_ratings = ratings_df[most_similar_user]

for movie, rating in similar_user_ratings.items():
    # Se o usuário similar gostou do filme (nota >= 4) e a Ana não viu (NaN)
    if rating >= 4 and pd.isna(ratings_df.loc[movie, target_user]):
        recommendations.append(movie)

print(f"\nRecomendação para {target_user} com base em {most_similar_user}: {recommendations}")

In [ ]:
"""
### Abordagem 2: Filtragem Colaborativa Baseada no Item (Item-Based)

**Objetivo**: Dado que a `Ana` gostou de 'Matrix', o que mais ela poderia gostar?

**Passos**:
1.  Encontrar o filme mais parecido com 'Matrix'.
2.  Recomendar esse filme para a `Ana` (se ela ainda não o viu).
"""

# Passo 1: Calcular a similaridade entre itens (filmes)
# Desta vez, calculamos a similaridade na matriz original (item-usuário)
item_similarity = cosine_similarity(ratings_filled)

# Transformando em um DataFrame
item_sim_df = pd.DataFrame(item_similarity, index=ratings_df.index, columns=ratings_df.index)

print("Matriz de Similaridade entre Itens (Filmes):")
print(item_sim_df)

# Passo 2: Fazer a recomendação com base em um filme
target_movie = 'Matrix'

# Encontra filmes similares (exceto ele mesmo)
similar_movies = item_sim_df[target_movie].sort_values(ascending=False)[1:]
print(f"\nFilmes mais similares a '{target_movie}':\n{similar_movies}")

# Pegando o filme mais similar
most_similar_movie = similar_movies.index[0]

# Verificando se a Ana já viu o filme mais similar
if pd.isna(ratings_df.loc[most_similar_movie, target_user]):
    print(f"\nRecomendação para {target_user}: Como você gostou de '{target_movie}', talvez goste de '{most_similar_movie}'.")
else:
    print(f"\nNão há novas recomendações baseadas em '{target_movie}', pois o filme mais similar ('{most_similar_movie}') já foi visto por {target_user}.")

In [ ]:
"""
### Conclusão

Neste notebook, implementamos duas abordagens fundamentais de filtragem colaborativa:

-   **User-Based CF**: Ótima para encontrar pessoas com gostos parecidos e explorar o que elas consomem. Pode sofrer com a **esparsidade dos dados** (quando os usuários avaliam poucos itens em comum).
-   **Item-Based CF**: Geralmente mais estável, pois a similaridade entre itens muda com menos frequência do que os gostos dos usuários. É a base de muitas recomendações de produtos ("*clientes que compraram X também compraram Y*").

Ambas as técnicas são a base para sistemas mais complexos. O próximo passo em seus estudos seria explorar métodos de **Fatoração de Matrizes (Matrix Factorization)**, como SVD, que são **modelos latentes** capazes de capturar características implícitas de usuários e itens para fazer previsões de notas mais precisas.
"""